# Second Order IIR Function
- $y_n+a_1y_{n-1}+a_2y_{n-2}=x_n+b_1x_{n-1}+b_2x_{N-2}$ 
- function
    - Scalar Filtering (benchmark)
    - Block Filtering
    - Multi-Block Filtering
        - PH Decomposition
        - Cyclic Reduction
- Function is writing in a SIMD way
    - avoid using special functions, e.g., np.inv 
    - easy to compare with C++ code
    - using numpy array to represent the SIMD block.

In [68]:
import numpy as np

In [69]:
# filter taps and initial states - LPF
b1 = 2
b2 = 1
a1 = -1.3
a2 = 0.4

xi1 = 2
xi2 = 1
yi1 = -3
yi2 = -5

# block size 
L = 8 

# number of blocks
N = 32

In [70]:
x = np.arange(N*L)

## Scalar Filtering
- $y_n=x_n+b_1x_{n-1}+b_2x_{N-2}-a_1y_{n-1}-a_2y_{n-2}$
- mainly for checking accuracy of complex algorithms

In [72]:
def scalar_filtering(x):
    
    y = np.zeros_like(x,dtype=float)
    for n in range(len(x)):
        if n == 0:
            y[0] = x[0] + b1*xi1 + b2*xi2 - a1*yi1 -a2* yi2
        elif n == 1:
            y[1] = x[1] + b1*x[0] + b2*xi1 - a1*y[0] - a2*yi1
        else:
            y[n] = x[n] + b1*x[n-1] + b2*x[n-2] - a1*y[n-1] - a2*y[n-2]

    return y

In [73]:
y_bench = scalar_filtering(x)

## Block Filtering
- block filtering can be represented by a state-space equation:
$$
\underbrace{
\begin{bmatrix}
 1 &      &       & &  \\
a_1 & 1 &       &  &  \\
a_2 & a_1 & \ddots &  &  \\
 & \ddots & \ddots & \ddots  &  \\
 &  & a_2 & a_1 & 1
\end{bmatrix}
}_{\mathbf{A}^{(L \times L)}}
\underbrace{
\begin{bmatrix} 
y_0  \\ y_1 \\ y_2 \\ \vdots \\ y_{L-1} 
\end{bmatrix}
}_{\mathbf{y_n}}
+
\underbrace{
\begin{bmatrix}
a_2 & a_1 \\
 & a_2     \\
 &   \\
 &   \\
 &  
\end{bmatrix}
}_{\mathbf{A}^{(L \times 2)}_-}
\underbrace{
\begin{bmatrix} 
y_{-2}  \\ y_{-1}  
\end{bmatrix} 
}_{\mathbf{y_{n-1}}}
=
\underbrace{
\begin{bmatrix}
1 &       &       &  &  \\
b_1 & 1 &      &  &  \\
b_2 & b_1 & 1 &  &  \\
 & \ddots & \ddots & \ddots &  \\
 &  & b_2 & b_1 & 1
\end{bmatrix}
}_{\mathbf{B}^{(L \times L)}}
\underbrace{
\begin{bmatrix} 
x_0  \\ x_1 \\ x_2 \\ \vdots \\ x_{L-1} 
\end{bmatrix}
}_{\mathbf{x_n}}
+
\underbrace{
\begin{bmatrix}
b_2 & b_1      \\
 & b_2 \\
 &   \\
 &  \\
 & 
\end{bmatrix}
}_{\mathbf{B}^{(L \times 2)}_-}
\underbrace{
\begin{bmatrix} 
x_{-2}  \\ x_{-1} 
\end{bmatrix} 
}_{\mathbf{x_{n-1}}} 
$$
The matrix form can be generalized to $n$ blocks using notations and simplified to:
$$
\begin{aligned}
   \mathbf{A}\mathbf{y_n} + \mathbf{A_-y_{n-1}}
     &= \mathbf{Bx_n} + \mathbf{B_-x_{n-1}}  \\
    \mathbf{y_n} &= \mathbf{A}^{-1}\mathbf{Bx_n} + \mathbf{A}^{-1}\mathbf{B_-x_{n-1}} + \mathbf{A}^{-1}\mathbf{A_-y_{n-1}} \\
    &\triangleq \mathbf{Hx_n}
    + \mathbf{H_{B_-}x_{n-1}}
    + \mathbf{H_{A_-}y_{n-1}}
\end{aligned}
$$
- $\mathbf{A}^{-1}$ can be derived using its reciprocal power series representation, where
$$
\mathbf{A}^{-1} = 
\begin{bmatrix}
s_0 &       &      & &  \\
s_1 & s_0 &      &  &  \\
s_2 & s_1 & s_0 &  &  \\
\vdots & \ddots & \ddots & \ddots &  \\
s_{L-1} & \cdots & s_2 & s_1 & s_0
\end{bmatrix}
$$
where $s_0 = 1$ and $s_n = \sum_{j=1}^{\min(n,2)} -a_j s_{n-j}$. Thus, $s_n$ is the truncated impulse response of the feedback (recurrence) part.
- $\mathbf{A}^{-1}\mathbf{B}$ is the matrix convolution between the truncated impulse response of feedback and feedforward parts, which is the total impulse response of the IIR filter. Thus, $\mathbf{H}=\mathbf{A}^{-1}\mathbf{B}$ is a also a lower triangular Toeplitz matrix, where the first column is the total impulse response.
- $\mathbf{A}^{-1}\mathbf{B_-}$ is a $L\times2$ matrix, where the first column is $b_2[s_0,s_1,...]$ and the second column is the convolution of  $b_1[s_0,s_1,...]+b_2[0,s_0,s_1,...]$. Similar to $\mathbf{A}^{-1}\mathbf{A_-}$.

In [75]:
def block_filtering_factors(b1,b2,a1,a2):
    
    # impulse response of recursive part, h0=[s_0,s_1...]
    h0 = np.zeros(L,dtype=float)
    h0[0] = 1
    h0[1] = -a1*h0[0]
    for n in range(2,L):
        h0[n] = -a1*h0[n-1] - a2*h0[n-2]

    # total impulse response h
    h = np.zeros(L,dtype=float)
    h[0] = 1
    h[1] = b1 - a1*h[0]
    h[2] = b2 - a1*h[1] - a2*h[0]  
    for n in range(3,L):
        h[n] = -a1*h[n-1] - a2*h[n-2]

    # filtering matrix H for x_n
    H = np.zeros((L,L),dtype=float)
    H[0] = h
    for n in range(1,L):
        H[n] = np.concatenate(([0],H[n-1][:-1])) # shuffle

    # filtering matrix HB for x_{n-1}
    HB = np.zeros((2,L),dtype=float)
    HB[0] = b2*h0
    HB[1] = b1*h0 + b2*np.concatenate(([0],h0[:-1]))

    # filtering matrix HA for y_{n-1}
    HA = np.zeros((2,L),dtype=float)
    HA[0] = a2*h0
    HA[1] = a1*h0 + a2*np.concatenate(([0],h0[:-1]))
    
    return H,HB,HA,h0

In [76]:
# check impulse response
from scipy.signal import lfilter

H,HB,HA,h0=block_filtering_factors(b1,b2,a1,a2)

impulse = np.zeros(L)
impulse[0] = 1
_h = lfilter([1,b1,b2],[1,a1,a2],impulse)

assert np.isclose(_h,H[0]).all()

_h0 = lfilter([1],[1,a1,a2],impulse)

assert np.isclose(_h0,h0).all()

In [77]:
# helper function
def start_register(xi1=0,xi2=0,yi1=0,yi2=0):

    buffer_x=np.zeros(L,dtype=float)
    buffer_y=np.zeros(L,dtype=float)
    
    buffer_x[L-2] = xi2
    buffer_x[L-1] = xi1
    buffer_y[L-2] = yi2
    buffer_y[L-1] = yi1

    return buffer_x,buffer_y

In [78]:
class block_filtering:

    def __init__(self,b1,b2,a1,a2,xi1,xi2,yi1,yi2):
        
        self.H,self.HB,self.HA,_=block_filtering_factors(b1,b2,a1,a2)
        self.buffer_x,self.buffer_y=start_register(xi1,xi2,yi1,yi2)
    
    def __call__(self,x):

        y = np.zeros_like(x,dtype=float)
        # adding Hx_n
        for n in range(L):
            y += self.H[n]*x[n]

        # adding HBx_{n-1} and HAy_{n-1}
        y += self.HB[0]*self.buffer_x[-2] + self.HB[1]*self.buffer_x[-1] - self.HA[0]*self.buffer_y[-2] - self.HA[1]*self.buffer_y[-1]

        # store x_{n-1},x_{n-2},y_{n-1},y_{n-2} in buffer
        self.buffer_x = x
        self.buffer_y = y

        return y

In [79]:
BF = block_filtering(b1,b2,a1,a2,xi1,xi2,yi1,yi2)

y_bf = np.zeros_like(x,dtype=float)
for n in range(N):
    y_bf[n*L:(n+1)*L] = BF(x[n*L:(n+1)*L])

assert np.isclose(y_bench,y_bf).all()
print('block filtering ok')

block filtering ok


## Multi-Block Filtering
- multi-block filtering essentially process multiple blocks in parallel. The matrix form of computing $NL$ samples is given by
$$ 
\begin{bmatrix}
1      &        &        &        &        &        \\
a_1    & 1      &        &        &        &        \\
a_2    & a_1    & 1      &        &        &        \\
       & a_2    & \ddots & \ddots &        &        \\
       &        & \ddots & \ddots & 1      &        \\
       &        &        & a_2    & a_1    & 1
\end{bmatrix}
\begin{bmatrix}
y_{0} \\
y_{1} \\
y_{2} \\
\vdots \\
y_{\mathbf{N}\mathbf{L}-2} \\
y_{\mathbf{N}\mathbf{L}-1}
\end{bmatrix}
=
\begin{bmatrix}
1      &        &        &        &        &        \\
b_1    & 1      &        &        &        &        \\
b_2    & b_1    & 1      &        &        &        \\
       & b_2    & \ddots & \ddots &        &        \\
       &        & \ddots & \ddots & 1      &        \\
       &        &        & b_2    & b_1    & 1
\end{bmatrix}
\begin{bmatrix}
x_{0} \\
x_{1} \\
x_{2} \\
\vdots \\
x_{\mathbf{N}\mathbf{L}-2} \\
x_{\mathbf{N}\mathbf{L}-1}
\end{bmatrix} 
+
\begin{bmatrix}
b_{2} & b_{1} \\
 & b_{2} \\
 & \\
 & \\
\end{bmatrix}
\begin{bmatrix}
x_{-2} \\
x_{-1} \\
\end{bmatrix}
- 
\begin{bmatrix}
a_{2} & a_{1} \\
 & a_{2} \\
 & \\
 & \\
\end{bmatrix}
\begin{bmatrix}
y_{-2} \\
y_{-1} \\
\end{bmatrix}
$$
Permute the samples in a space of $N$ and group all $L$ samples, the matrix form represented by blocks now becomes
$$
\begin{bmatrix}
\mathbf{I}      &        &        &        &    a_{2}\mathbf{R}    &     a_{1}\mathbf{R}   \\
a_{1}\mathbf{I}    & \mathbf{I}      &        &        &        &   a_{2}\mathbf{R}     \\
a_{2}\mathbf{I}    & a_{1}\mathbf{I}    & \mathbf{I}      &        &        &        \\
       & a_{2}\mathbf{I}    & \ddots & \ddots &        &        \\
       &        & \ddots & \ddots & \mathbf{I}      &        \\
       &        &        & a_{2}\mathbf{I}    & a_{1}\mathbf{I}    & \mathbf{I}
\end{bmatrix}
\begin{bmatrix}
\mathbf{y_{0}} \\
\mathbf{y_{1}} \\
\mathbf{y_{2}} \\
\vdots \\
\mathbf{y_{N-2}} \\
\mathbf{y_{N-1}}
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{I}      &        &        &        &    b_{2}\mathbf{R}    &     b_{1}\mathbf{R}   \\
b_{1}\mathbf{I}    & \mathbf{I}      &        &        &        &   b_{2}\mathbf{R}     \\
b_{2}\mathbf{I}    & b_{1}\mathbf{I}    & \mathbf{I}      &        &        &        \\
       & b_{2}\mathbf{I}    & \ddots & \ddots &        &        \\
       &        & \ddots & \ddots & \mathbf{I}      &        \\
       &        &        & b_{2}\mathbf{I}    & b_{1}\mathbf{I}    & \mathbf{I}
\end{bmatrix}
\begin{bmatrix}
\mathbf{x_{0}} \\
\mathbf{x_{1}} \\
\mathbf{x_{2}} \\
\vdots \\
\mathbf{x_{N-2}} \\
\mathbf{x_{N-1}}
\end{bmatrix}
+
\begin{bmatrix}
b_{2}\mathbf{s} & b_{1}\mathbf{s} \\
 & b_{2}\mathbf{s} \\
 & \\
 & \\
\end{bmatrix}
\begin{bmatrix}
x_{-2} \\
x_{-1} \\
\end{bmatrix}
-
\begin{bmatrix}
a_{2}\mathbf{s} & a_{1}\mathbf{s} \\
 & a_{2}\mathbf{s} \\
 & \\
 & \\
\end{bmatrix}
\begin{bmatrix}
y_{-2} \\
y_{-1} \\
\end{bmatrix}
$$ 
where $\mathbf{x_{n}}=[x_{n}, x_{n+\mathbf{N}}, \cdots, x_{n+(\mathbf{L}-1)\mathbf{N}}]^T$. $\mathbf{R}$ is a lower shift matrix of size $\mathbf{L}$ and $\mathbf{s}$ is the vector of size $\mathbf{L}$ with only the first element 1,
$$
\mathbf{R}=
\begin{bmatrix}
 &  &  &  \\
1 &  &  &  \\
 & \ddots &  &  \\
 &  & 1 &  \\
\end{bmatrix}, \quad
\mathbf{S}=
\begin{bmatrix}
1  \\
   \\
   \\ 
   \\
\end{bmatrix}
$$

### Permutation
Let us first discuss block permutation, that is the first step for multi-block filtering.
Furthermore, we are doing rectangular matrix permutation since the dimension is $\mathbf{L}\times\mathbf{N}$, and $\mathbf{N}$ is mostly chosen a large value to have a better performance. Take $\mathbf{L}=8$ and $\mathbf{N}=16$ as an example.

Before permutation:
$$
\mathbf{X} =
\left[
\begin{array}{cccccccc:cccccccc}
0 & 8  & 16 & 24 & 32 & 40 & 48 & 56 &    64 & 72 & 80 & 88 & 96 & 104 & 112 & 120 \\
1 & 9  & 17 & 25 & 33 & 41 & 49 & 57 &    65 & 73 & 81 & 89 & 97 & 105 & 113 & 121 \\
2 & 10 & 18 & 26 & 34 & 42 & 50 & 58 &    66 & 74 & 82 & 90 & 98 & 106 & 114 & 122 \\
3 & 11 & 19 & 27 & 35 & 43 & 51 & 59 &    67 & 75 & 83 & 91 & 99 & 107 & 115 & 123 \\
\hdashline
4 & 12 & 20 & 28 & 36 & 44 & 52 & 60 &    68 & 76 & 84 & 92 &100 & 108 & 116 & 124 \\
5 & 13 & 21 & 29 & 37 & 45 & 53 & 61 &    69 & 77 & 85 & 93 &101 & 109 & 117 & 125 \\
6 & 14 & 22 & 30 & 38 & 46 & 54 & 62 &    70 & 78 & 86 & 94 &102 & 110 & 118 & 126 \\
7 & 15 & 23 & 31 & 39 & 47 & 55 & 63 &    71 & 79 & 87 & 95 &103 & 111 & 119 & 127 \\
\end{array}
\right]
$$
where each column corresponds to a block. Every time the permutation is performed by switching the lower-left corner and upper-right corner, as illustrated in dashed line. Furthermore, the order of the new columns from left to right is [top n,top n+N/2], [bottom n,bottom n+N/2], 
[top n+1,top n+1+N/2)], [bottom n+1,bottom n+1+N/2)] and so on. 

---

After first-time permutation:
$$
\left[
\begin{array}{cccccccc:cccccccc}
0  & 4  & 8  & 12 & 16 & 20 & 24 & 28 & 32 & 36 & 40 & 44 & 48 & 52 & 56 & 60 \\
1  & 5  & 9  & 13 & 17 & 21 & 25 & 29 & 33 & 37 & 41 & 45 & 49 & 53 & 57 & 61 \\
\hdashline
2  & 6  & 10 & 14 & 18 & 22 & 26 & 30 & 34 & 38 & 42 & 46 & 50 & 54 & 58 & 62 \\
3  & 7  & 11 & 15 & 19 & 23 & 27 & 31 & 35 & 39 & 43 & 47 & 51 & 55 & 59 & 63 \\
\hline
64 & 68 & 72 & 76 & 80 & 84 & 88 & 92 & 96 & 100 & 104 & 108 & 112 & 116 & 120 & 124 \\
65 & 69 & 73 & 77 & 81 & 85 & 89 & 93 & 97 & 101 & 105 & 109 & 113 & 117 & 121 & 125 \\
\hdashline
66 & 70 & 74 & 78 & 82 & 86 & 90 & 94 & 98 & 102 & 106 & 110 & 114 & 118 & 122 & 126 \\
67 & 71 & 75 & 79 & 83 & 87 & 91 & 95 & 99 & 103 & 107 & 111 & 115 & 119 & 123 & 127 \\
\end{array}
\right]
$$
Thus the top half and bottom half are well separated. Then the next step is to switch the lower-left corner and upper-right corner on top and bottom, respectively. Regroup the new columns as previous.

---

After second-time permutation:
$$
\left[
\begin{array}{cccccccc:cccccccc}
0  & 2  & 4  & 6  & 8  & 10 & 12 & 14 & 16 & 18 & 20 & 22 & 24 & 26 & 28 & 30 \\
\hdashline
1  & 3  & 5  & 7  & 9  & 11 & 13 & 15 & 17 & 19 & 21 & 23 & 25 & 27 & 29 & 31 \\
\hline
32 & 34 & 36 & 38 & 40 & 42 & 44 & 46 & 48 & 50 & 52 & 54 & 56 & 58 & 60 & 62 \\
\hdashline
33 & 35 & 37 & 39 & 41 & 43 & 45 & 47 & 49 & 51 & 53 & 55 & 57 & 59 & 61 & 63 \\
\hline
64 & 66 & 68 & 70 & 72 & 74 & 76 & 78 & 80 & 82 & 84 & 86 & 88 & 90 & 92 & 94 \\
\hdashline
65 & 67 & 69 & 71 & 73 & 75 & 77 & 79 & 81 & 83 & 85 & 87 & 89 & 91 & 93 & 95 \\
\hline
96 & 98 &100 &102 &104 &106 &108 &110 &112 &114 &116 &118 &120 &122 &124 &126 \\
\hdashline
97 & 99 &101 &103 &105 &107 &109 &111 &113 &115 &117 &119 &121 &123 &125 &127 \\
\end{array}
\right]
$$
After second-time permutation, every two rows are well separated. Follow the same rule, 

---

after third-time permutation:
$$
\mathbf{X^P}=
\left[
\begin{array}{cccccccccccccccc}
0   & 1   & 2   & 3   & 4   & 5   & 6   & 7   & 8   & 9   & 10  & 11  & 12  & 13  & 14  & 15 \\
16  & 17  & 18  & 19  & 20  & 21  & 22  & 23  & 24  & 25  & 26  & 27  & 28  & 29  & 30  & 31 \\
32  & 33  & 34  & 35  & 36  & 37  & 38  & 39  & 40  & 41  & 42  & 43  & 44  & 45  & 46  & 47 \\
48  & 49  & 50  & 51  & 52  & 53  & 54  & 55  & 56  & 57  & 58  & 59  & 60  & 61  & 62  & 63 \\
64  & 65  & 66  & 67  & 68  & 69  & 70  & 71  & 72  & 73  & 74  & 75  & 76  & 77  & 78  & 79 \\
80  & 81  & 82  & 83  & 84  & 85  & 86  & 87  & 88  & 89  & 90  & 91  & 92  & 93  & 94  & 95 \\
96  & 97  & 98  & 99  & 100 & 101 & 102 & 103 & 104 & 105 & 106 & 107 & 108 & 109 & 110 & 111 \\
112 & 113 & 114 & 115 & 116 & 117 & 118 & 119 & 120 & 121 & 122 & 123 & 124 & 125 & 126 & 127 \\
\end{array}
\right]
$$
Thus, the permutation of a $\mathbf{L}\times\mathbf{N}$ matrix requires $\mathbf{N}\log_2(\mathbf{L})$ shuffles.

In [82]:
def permute_V8(X):

    tmp1 = np.zeros_like(X)
    tmp2 = np.zeros_like(X)
    X_T = np.zeros_like(X)

    # Stage 1: Blend top and bottom halves (step size 8)
    for n in range(N // 2):
        a = X[n]
        b = X[n + N // 2]

        tmp1[2 * n] = np.concatenate([a[0:4], b[0:4]])         # 0,1,2,3 | 8,9,10,11
        tmp1[2 * n + 1] = np.concatenate([a[4:8], b[4:8]])      # 4,5,6,7 | 12,13,14,15

    # Stage 2: Blend within tmp1 (L/2 × N)
    for n in range(N // 2):
        a = tmp1[n]
        b = tmp1[n + N // 2]

        tmp2[2 * n] = np.concatenate([a[0:2], b[0:2], a[4:6], b[4:6]])     # 0,1 | 8,9 | 4,5 | 12,13
        tmp2[2 * n + 1] = np.concatenate([a[2:4], b[2:4], a[6:8], b[6:8]]) # 2,3 | 10,11 | 6,7 | 14,15

    # Stage 3: Final blend (L/4 × N)
    for n in range(N // 2):
        a = tmp2[n]
        b = tmp2[n + N // 2]

        X_T[2 * n] = np.concatenate([[a[0]], [b[0]], [a[2]], [b[2]], [a[4]], [b[4]], [a[6]], [b[6]]])
        X_T[2 * n + 1] = np.concatenate([[a[1]], [b[1]], [a[3]], [b[3]], [a[5]], [b[5]], [a[7]], [b[7]]])

    return X_T

### De-permutation

Let us now discuss **block de-permutation**, which reverses the transformation applied by the three-stage permutation for multi-block filtering. The de-permutation also operates on a rectangular matrix of dimension $\mathbf{L} \times \mathbf{N}$, and we again take $\mathbf{L} = 8$ and $\mathbf{N} = 16$ as an example.

Before de-permutation (i.e., after full permutation):

$$
\mathbf{X}^P =
\left[
\begin{array}{cccc:cccc|cccc:cccc}
0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 \\
16 & 17 & 18 & 19 & 20 & 21 & 22 & 23 & 24 & 25 & 26 & 27 & 28 & 29 & 30 & 31 \\
32 & 33 & 34 & 35 & 36 & 37 & 38 & 39 & 40 & 41 & 42 & 43 & 44 & 45 & 46 & 47 \\
48 & 49 & 50 & 51 & 52 & 53 & 54 & 55 & 56 & 57 & 58 & 59 & 60 & 61 & 62 & 63 \\
\hdashline
64 & 65 & 66 & 67 & 68 & 69 & 70 & 71 & 72 & 73 & 74 & 75 & 76 & 77 & 78 & 79 \\
80 & 81 & 82 & 83 & 84 & 85 & 86 & 87 & 88 & 89 & 90 & 91 & 92 & 93 & 94 & 95 \\
96 & 97 & 98 & 99 & 100 & 101 & 102 & 103 & 104 & 105 & 106 & 107 & 108 & 109 & 110 & 111 \\
112 & 113 & 114 & 115 & 116 & 117 & 118 & 119 & 120 & 121 & 122 & 123 & 124 & 125 & 126 & 127
\end{array}
\right]
$$

---

Similar as the step in permute, switch the lower left and upper right corner that are seperated by dashed line. But the order of new columns from left to right is different from permute, that is [top n,top n+N/4], [top n+N/2,top n+N/2+N/4], ...,
[top n+1,top n+1+N/4], [top n+1+N/2,top n+1+N/2], ... [bottom n,bottom n+N/4] and so on. 

After first-time de-permutation:

$$
\left[
\begin{array}{cccc:cccc|cccc:cccc}
0  & 8  & 1 & 9 & 2 & 10 & 3 & 11 & 64 & 72 & 65 & 73 & 66 & 74 & 67 & 75 \\
16 & 24 & 17 & 25 & 18 & 26 & 19 & 27 & 80 & 88 & 81 & 89 & 82 & 90 & 83 & 91 \\
\hdashline
32 & 40 & 33 & 41 & 34 & 42 & 35 & 43 & 96 & 104 & 97 & 105 & 98 & 106 & 99 & 107 \\
48 & 56 & 49 & 57 & 50 & 58 & 51 & 59 & 112 & 120 & 113 & 121 & 114 & 122 & 115 & 123 \\
\hline
4 & 12 & 5 & 13 & 6 & 14 & 7 & 15 & 68 & 76 & 69 & 77 & 70 & 78 & 71 & 79 \\
20 & 28 & 21 & 29 & 22 & 30 & 23 & 31 & 84 & 92 & 85 & 93 & 86 & 94 & 87 & 95 \\
\hdashline
36 & 44 & 37 & 45 & 38 & 46 & 39 & 47 & 100 & 108 & 101 & 109 & 102 & 110 & 103 & 111 \\
52 & 60 & 53 & 61 & 54 & 62 & 55 & 63 & 116 & 124 & 117 & 125 & 118 & 126 & 119 & 127
\end{array}
\right]
$$

---

Now the left half and right half matrix are well seperated. Then permute in submatrices in each half matrices, no reorder is needed.

After second-time permutation:

$$
\left[
\begin{array}{cc:cc|cc:cc|cc:cc|cc:cc}
0  & 8  & 1 & 9 & 32 & 40 & 33 & 41 &       64 & 72 & 65 & 73 & 66 & 74 & 67 & 75 \\
\hdashline
16 & 24 & 17 & 25 & 48 & 56 & 49 & 57 &   80 & 88 & 81 & 89 & 82 & 90 & 83 & 91 \\
\hline
2 & 10 & 3 & 11 & 34 & 42 & 35 & 43 &   96 & 104 & 97 & 105 & 98 & 106 & 99 & 107 \\
\hdashline
18 & 26 & 19 & 27 & 50 & 58 & 51 & 59 &   112 & 120 & 113 & 121 & 114 & 122 & 115 & 123 \\
\hline
4 & 12 & 5 & 13 & 36 & 44 & 37 & 45 &       68 & 76 & 69 & 77 & 70 & 78 & 71 & 79 \\
\hdashline
20 & 28 & 21 & 29 & 52 & 60 & 53 & 61 &   84 & 92 & 85 & 93 & 86 & 94 & 87 & 95 \\
\hline
 6 & 14 & 7 & 15 & 38 & 46 & 39 & 47 &   100 & 108 & 101 & 109 & 102 & 110 & 103 & 111 \\
 \hdashline
22 & 30 & 23 & 31 & 54 & 62 & 55 & 63 &   116 & 124 & 117 & 125 & 118 & 126 & 119 & 127
\end{array}
\right]
$$

---

After third-time permutation:

$$
\mathbf{X} =
\begin{bmatrix}
0 & 8 & 16 & 24 & 32 & 40 & 48 & 56 & 64 & 72 & 80 & 88 & 96 & 104 & 112 & 120 \\
1 & 9 & 17 & 25 & 33 & 41 & 49 & 57 & 65 & 73 & 81 & 89 & 97 & 105 & 113 & 121 \\
2 & 10 & 18 & 26 & 34 & 42 & 50 & 58 & 66 & 74 & 82 & 90 & 98 & 106 & 114 & 122 \\
3 & 11 & 19 & 27 & 35 & 43 & 51 & 59 & 67 & 75 & 83 & 91 & 99 & 107 & 115 & 123 \\
4 & 12 & 20 & 28 & 36 & 44 & 52 & 60 & 68 & 76 & 84 & 92 & 100 & 108 & 116 & 124 \\
5 & 13 & 21 & 29 & 37 & 45 & 53 & 61 & 69 & 77 & 85 & 93 & 101 & 109 & 117 & 125 \\
6 & 14 & 22 & 30 & 38 & 46 & 54 & 62 & 70 & 78 & 86 & 94 & 102 & 110 & 118 & 126 \\
7 & 15 & 23 & 31 & 39 & 47 & 55 & 63 & 71 & 79 & 87 & 95 & 103 & 111 & 119 & 127
\end{bmatrix}
$$
Thus, the de-permutation of a $\mathbf{L} \times \mathbf{N}$ matrix requires $\mathbf{N} \log_2(\mathbf{L})$ shuffles.

In [84]:
def depermute_V8(X_T):

    tmp1 = np.zeros_like(X_T)
    tmp2 = np.zeros_like(X_T)
    X = np.zeros_like(X_T)

    # Stage 1: Reverse last-stage 1-element interleave (L/4 x N)
    for l in range(4):
        for n in range(N // 8):
            a = X_T[l + 8 * n]
            b = X_T[l + 4 + 8 * n]
            tmp1[l * (N // 8) + n] = np.concatenate([a[0:4], b[0:4]])
            tmp1[l * (N // 8) + n + N // 2] = np.concatenate([a[4:8], b[4:8]])

    # Stage 2: Reverse middle-stage 2-element interleave (L/2 x N/2)
    for l in range(2):
        for n in range(N // 4):
            a = tmp1[n + l * (N // 2)]
            b = tmp1[n + l * (N // 2) + N // 4]
            tmp2[n + l * (N // 2)] =  np.concatenate([a[0:2], b[0:2], a[4:6], b[4:6]])     # 0,1 | 8,9 | 4,5 | 12,13
            tmp2[n + l * (N // 2) + N // 4] = np.concatenate([a[2:4], b[2:4], a[6:8], b[6:8]]) # 2,3 | 10,11 | 6,7 | 14,15

    # Stage 3: Reverse top-level 4-element interleave (L x N)
    for l in range(4):
        for n in range(N // 8):
            a = tmp2[n + l * (N // 4)]
            b = tmp2[n + l * (N // 4) + N // 8]
            X[n + l * (N // 4)] = np.concatenate([[a[0]], [b[0]], [a[2]], [b[2]], [a[4]], [b[4]], [a[6]], [b[6]]])
            X[n + l * (N // 4) + N // 8] = np.concatenate([[a[1]], [b[1]], [a[3]], [b[3]], [a[5]], [b[5]], [a[7]], [b[7]]])

    return X

In [85]:
X = []

for n in range(N):
    X.append(x[n*L:(n+1)*L])

assert np.isclose(X,depermute_V8(permute_V8(X))).all()
print('permute ok')

permute ok


### Solution for Non-Recursive Part
We then look at the non-recursive part ($\mathbf{v}$) in terms of blocks as
$$
\begin{aligned}
\mathbf{v}_0 &= \mathbf{x}_0 + b_1\mathbf{R}\mathbf{x_{N-1}} + b_2\mathbf{R}\mathbf{x_{N-2}} + b_2\mathbf{s}\mathbf{x_{-2}} + b_1\mathbf{s}\mathbf{x_{-1}} = \mathbf{x}_0 + b_1\mathbf{x_{-1}} + b_2\mathbf{x_{-2}} \\
\mathbf{v}_1 &= \mathbf{x}_1 + b_1\mathbf{x}_0 + b_2\mathbf{R}\mathbf{x_{N-1}} + b_2\mathbf{s}\mathbf{x_{-1}} = \mathbf{x}_1 + b_1\mathbf{x}_0 + b_2\mathbf{x_{-1}} \\
\mathbf{v}_2 &= \mathbf{x}_2 + b_1\mathbf{x}_1 + b_2\mathbf{x}_0
\end{aligned}
$$ 
where $\mathbf{y_{-2}}=[y_{-2},y_{\mathbf{N}-2},\cdots,y_{(\mathbf{L}-1)\mathbf{N}-2}]^{T}$ and $\mathbf{y_{-1}}=[y_{-1},y_{\mathbf{N}-1},\cdots,y_{(\mathbf{L}-1)\mathbf{N}-1}]^{T}$, which requires one shuffles each.

In [87]:
class non_recursive_solution:

    def __init__(self,b1,b2,xi1,xi2):
        
        self.buffer_x,_=start_register(xi1,xi2)
        self.b1=b1
        self.b2=b2

    def __call__(self,X):
    
        # prepare blocks x_{-1}, x_{-2}
        xvi2 = np.concatenate(([self.buffer_x[-2]],X[-2][:-1]),dtype=float)
        xvi1 = np.concatenate(([self.buffer_x[-1]],X[-1][:-1]),dtype=float)
        
        V = np.zeros_like(X,dtype=float)
    
        V[0] = X[0] + self.b1*xvi1 + self.b2*xvi2
        V[1] = X[1] + self.b1*X[0] + self.b2*xvi1    
        for n in range(2,len(X)):
            V[n] = X[n] + self.b1*X[n-1] + self.b2*X[n-2]

        self.buffer_x[-2] = X[-2][-1]
        self.buffer_x[-1] = X[-1][-1]
    
        return V

### Solution I for Recursive Part: (Particular and Homogeneous Solution) PH Decomposition
After adding the non-recursive part, the matrix form of multi-block filtering now becomes

$$
\begin{aligned}
\begin{bmatrix}
\mathbf{I}      &        &        &        &    a_{2}\mathbf{R}    &     a_{1}\mathbf{R}   \\
a_{1}\mathbf{I}    & \mathbf{I}      &        &        &        &   a_{2}\mathbf{R}     \\
a_{2}\mathbf{I}    & a_{1}\mathbf{I}    & \mathbf{I}      &        &        &        \\
       & a_{2}\mathbf{I}    & \ddots & \ddots &        &        \\
       &        & \ddots & \ddots & \mathbf{I}      &        \\
       &        &        & a_{2}\mathbf{I}    & a_{1}\mathbf{I}    & \mathbf{I}
\end{bmatrix}
\begin{bmatrix}
\mathbf{y_{0}} \\
\mathbf{y_{1}} \\
\mathbf{y_{2}} \\
\vdots \\
\mathbf{y_{N-2}} \\
\mathbf{y_{N-1}}
\end{bmatrix}
&=
\begin{bmatrix}
\mathbf{v_{0}} \\
\mathbf{v_{1}} \\
\mathbf{v_{2}} \\
\vdots \\
\mathbf{v_{N-2}} \\
\mathbf{v_{N-1}}
\end{bmatrix}
-
\begin{bmatrix}
a_{2}\mathbf{s} & a_{1}\mathbf{s} \\
 & a_{2}\mathbf{s} \\
 & \\
 & \\
\end{bmatrix}
\begin{bmatrix}
y_{-2} \\
y_{-1} \\
\end{bmatrix} \\
\mathbf{A}\mathbf{Y} &= \mathbf{V} -
\begin{bmatrix}
a_{2}\mathbf{s} & a_{1}\mathbf{s} \\
 & a_{2}\mathbf{s} \\
 & \\
 & \\
\end{bmatrix}
\begin{bmatrix}
y_{-2} \\
y_{-1} \\
\end{bmatrix}
\end{aligned}
$$ 
In PH decomposition, $\mathbf{A}$ can be decomposed as 
$$
\mathbf{A}=\mathbf{P}\mathbf{H}=\mathbf{P}
\left[
\begin{array}{c: c}
\mathbf{H_{11}} & \mathbf{H_{12}} \\
\hdashline
\mathbf{H_{21}} & \mathbf{H_{22}}
\end{array}
\right]
$$
where $\mathbf{P}\mathbf{W}=\mathbf{V}$, $\mathbf{H}\mathbf{Y}=\mathbf{W}-\mathbf{P}^{-1}\mathbf{Y_{-}}$ denote the particular solution and homogeneous solution, respectively, and 
$$
\mathbf{P} =
\begin{bmatrix}
\mathbf{I}             &                   &                   &                   &                   \\
a_{1}\mathbf{I}        & \mathbf{I}        &                   &                   &                   \\
a_{2}\mathbf{I}        & a_{1}\mathbf{I}   & \ddots        &                   &                   \\
                       & \ddots   & \ddots   & \ddots        &                   \\
                       &                   & a_{2}\mathbf{I}   & a_{1}\mathbf{I}   & \mathbf{I}
\end{bmatrix}, \quad
\mathbf{H} = \left[
\begin{array}{cccc:cc}
\mathbf{I} & & & & \mathbf{u_{2,0}} & \mathbf{u_{1,0}} \\
 & \mathbf{I} & & & \mathbf{u_{2,1}} &  \mathbf{u_{1,1}} \\
 & & \ddots & & \vdots & \vdots \\
 & & & \mathbf{I} & \mathbf{u_{2,N-3}} &  \mathbf{u_{1,N-3}} \\
 \hdashline
 & & & & \mathbf{I}+\mathbf{u_{2,N-2}} & \mathbf{u_{1,N-2}} \\
 & & & & \mathbf{u_{2,N-1}} & \mathbf{I}+\mathbf{u_{1,N-1}}
\end{array} \right]
$$
where $\mathbf{u_{2,0}}=a_{2}\mathbf{R}$, 
$\mathbf{u_{1,0}}=a_{1}\mathbf{R}$, $\mathbf{u_{2,1}}=-a_{1}a_{2}\mathbf{R}$, 
$\mathbf{u_{1,1}}=(a_{2}-a_{1}^2)\mathbf{R}$,
and $\mathbf{u_{i,j}}=-a_{2}\mathbf{u_{i,j-2}}-a_{1}\mathbf{u_{i,j-1}}$.

#### Paricular Solution
The complete recursive part can be represented by
$$
\mathbf{P}\mathbf{H}\mathbf{Y}=\mathbf{V}-\mathbf{Y_{-}}
$$
where $\mathbf{Y_{-}}$ include the initial states. Move $\mathbf{P}$ to the right-hand side,
$$
\mathbf{H}\mathbf{Y}=\mathbf{P}^{-1}\mathbf{V}-\mathbf{P}^{-1}\mathbf{Y_{-}}
$$
The particular solution is to solve $\mathbf{P}\mathbf{W}=\mathbf{V}$ and the homogeneous solution is to compute $\mathbf{Y}$ from the rest $\mathbf{H}\mathbf{Y}+\mathbf{P}^{-1}\mathbf{Y_{-}}=\mathbf{W}$

In [90]:
def particular_solution(V,a1,a2):

    # solve pw = v, and p is a the summation of an identity matrix and lower bidiagonal.
    W = np.zeros_like(V,dtype=float)

    W[0] = V[0]
    W[1] = V[1] - a1*W[0]
    for n in range(2,len(V)):
        W[n] = V[n] - a1*W[n-1] - a2*W[n-2]

    return W

#### Homogeneous Solution
The equation of homogeneous section is given by 
$$
\mathbf{H}\mathbf{Y}+\mathbf{P}^{-1}\begin{bmatrix}
a_{2}\mathbf{s} & a_{1}\mathbf{s} \\
 & a_{2}\mathbf{s} \\
 & \\
 & \\
\end{bmatrix}
\begin{bmatrix}
y_{-2} \\
y_{-1} \\
\end{bmatrix}=\mathbf{W}.
$$
It can be further reduced to
$$
\mathbf{H}
\mathbf{Y} + \mathbf{Q}
\begin{bmatrix}
y_{-2} \\
y_{-1} \\
\end{bmatrix}  = \mathbf{W} 
$$
where 
$$
\mathbf{Q} = 
\begin{bmatrix}
\mathbf{q_{2,0}} &  \mathbf{q_{1,0}} \\
\mathbf{q_{2,1}} &  \mathbf{q_{1,1}} \\
\vdots & \vdots \\
\mathbf{q_{2,N-1}} &  \mathbf{q_{1,N-1}} \\
\end{bmatrix}
$$
and $\mathbf{q_{2,0}}=a_{2}\mathbf{s}$, 
$\mathbf{q_{1,0}}=a_{1}\mathbf{s}$, $\mathbf{q_{2,1}}=-a_{1}a_{2}\mathbf{s}$, 
$\mathbf{q_{1,1}}=(a_{2}-a_{1}^2)\mathbf{s}$,
$\mathbf{q_{i,j}}=-a_{2}\mathbf{q_{i,j-2}}-a_{1}\mathbf{q_{i,j-1}}$. Thus, $\mathbf{q}_{i,j}$ have the same scalar coefficients as $\mathbf{u}_{i,j}$ and block filtering factor $\mathbf{H_{A_-}}$ but extended to $N$.

Let's write again the detailed version of the equation,
$$
\begin{aligned}
\mathbf{H}
\mathbf{Y} + \mathbf{Q}
\begin{bmatrix}
y_{-2} \\
y_{-1} \\
\end{bmatrix}  &= \mathbf{W}  \\
\left[
\begin{array}{cccc:cc}
\mathbf{I} & & & & \mathbf{u_{2,0}} & \mathbf{u_{1,0}} \\
 & \mathbf{I} & & & \mathbf{u_{2,1}} &  \mathbf{u_{1,1}} \\
 & & \ddots & & \vdots & \vdots \\
 & & & \mathbf{I} & \mathbf{u_{2,N-3}} &  \mathbf{u_{1,N-3}} \\
 \hdashline
 & & & & \mathbf{I}+\mathbf{u_{2,N-2}} & \mathbf{u_{1,N-2}} \\
 & & & & \mathbf{u_{2,N-1}} & \mathbf{I}+\mathbf{u_{1,N-1}}
\end{array} \right]
\begin{bmatrix}
\mathbf{y_{0}} \\
\mathbf{y_{1}} \\
\mathbf{y_{2}} \\
\vdots \\
\mathbf{y_{N-2}} \\
\mathbf{y_{N-1}}
\end{bmatrix}
+ 
\begin{bmatrix}
\mathbf{q_{2,0}} &  \mathbf{q_{1,0}} \\
\mathbf{q_{2,1}} &  \mathbf{q_{1,1}} \\
\vdots & \vdots \\
\mathbf{q_{2,N-1}} &  \mathbf{q_{1,N-1}} \\
\end{bmatrix}
\begin{bmatrix}
y_{-2} \\
y_{-1} \\
\end{bmatrix} &= \mathbf{W}
\end{aligned}
$$

To compute the output blocks, we need to first compute the last two blocks, that is 
$$
\begin{aligned}
\begin{bmatrix}
 \mathbf{I}+\mathbf{u_{2,N-2}} & \mathbf{u_{1,N-2}} \\
 \mathbf{u_{2,N-1}} & \mathbf{I}+\mathbf{u_{1,N-1}}
\end{bmatrix}
\begin{bmatrix}
\mathbf{y_{N-2}} \\
\mathbf{y_{N-1}}
\end{bmatrix} 
+
\begin{bmatrix}
\mathbf{q_{2,N-2}} &  \mathbf{q_{1,N-2}} \\
\mathbf{q_{2,N-1}} &  \mathbf{q_{1,N-1}} \\
\end{bmatrix}
\begin{bmatrix}
y_{-2} \\
y_{-1} \\
\end{bmatrix} &= 
\begin{bmatrix}
\mathbf{w_{N-2}} \\
\mathbf{w_{N-1}}
\end{bmatrix}  \\
\left[
\begin{array}{cccc:cccc}
 1 &   &   &   &   &   &   &   \\
 e &  1 &   &   & g  &   &   &   \\
  & \ddots  & \ddots  &   &   & \ddots  &   &   \\
  &   & e  & 1  &   &   & g  &   \\
\hdashline
  &   &   &   & 1  &   &   &   \\
 f &   &   &   & d  & 1  &   &   \\
  & \ddots  &   &   &   & \ddots  & \ddots  &   \\
  &   & f  &   &   &   & d  & 1 
\end{array}
\right]
\begin{bmatrix}
 y_{\mathbf{N}-2} \\ y_{2\mathbf{N}-2} \\ \vdots \\ y_{\mathbf{L}\mathbf{N}-2} \\
\hdashline
 y_{\mathbf{N}-1} \\ y_{2\mathbf{N}-1} \\ \vdots \\ y_{\mathbf{L}\mathbf{N}-1} 
\end{bmatrix}
+
\begin{bmatrix}
 e & g \\
 & \\ 
 & \\ 
 & \\ 
\hdashline
 f & d \\ 
 &     \\ 
 &     \\
 &     \\
\end{bmatrix}
\begin{bmatrix}
y_{-2} \\
y_{-1} \\
\end{bmatrix} &=
\begin{bmatrix}
 w_{\mathbf{N}-2} \\ w_{2\mathbf{N}-2} \\ \vdots \\ w_{\mathbf{L}\mathbf{N}-2} \\
\hdashline
 w_{\mathbf{N}-1} \\ w_{2\mathbf{N}-1} \\ \vdots \\ w_{\mathbf{L}\mathbf{N}-1} 
\end{bmatrix}
\end{aligned}
$$
where $e$, $g$, $f$, and $d$ represent the scalar coefficient in $\mathbf{u_{2,N-2}}$, $\mathbf{u_{1,N-2}}$, $\mathbf{u_{2,N-1}}$, $\mathbf{u_{1,N-1}}$.
Then permute and regroup adjacent samples,
$$
\begin{bmatrix}
\mathbf{I}  &              &             &    \\
\mathbf{C}  & \mathbf{I}   &             &    \\
            & \ddots   & \ddots  &    \\
            &              & \mathbf{C}  & \mathbf{I}
\end{bmatrix}
\begin{bmatrix}
\mathbf{\bar{y}_{N-2}} \\ 
\mathbf{\bar{y}_{2N-2}} \\ 
\vdots \\ 
\mathbf{\bar{y}_{LN-2}} 
\end{bmatrix}
+
\begin{bmatrix}
\mathbf{C} \\
\\
\\
\\
\end{bmatrix}
\begin{bmatrix}
y_{-2} \\
y_{-1}
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{\bar{w}_{N-2}} \\ 
\mathbf{\bar{w}_{2N-2}} \\ 
\vdots \\ 
\mathbf{\bar{w}_{LN-2}} 
\end{bmatrix}
$$
where 
$$
\mathbf{C}=
\begin{bmatrix}
e   & g   \\
f  & d 
\end{bmatrix}
$$ 
and $\mathbf{\bar{y}_{n}}=[y_{n}, y_{n+1}]$ and $\mathbf{\bar{w}_{n}}=[w_{n}, w_{n+1}]$. Thus, the last two blocks can be computed by recursive doubling as its inner blocks satisfy
$$
\mathbf{\bar{y}_{n+N}}=
\mathbf{\bar{w}_{n+N}}-\mathbf{C}\mathbf{\bar{y}_{n}}
$$
Thus, in recursive doubling, the multiplier is $-\mathbf{C}$.

##### Block Recursive Doubling for solving the last two blocks
<div align="center">
  <img src="recursive_doubling.png" alt="Recursive Doubling Diagram" width="600"/>
</div>
The figure shows how to perform block recursive doubling for block size $L=8$. It should be noted that the samples $y_{N-2},y_{2N-2},\cdots,y_{8N-2}$ and $y_{N-1},y_{2N-1},\cdots,y_{8N-1}$ are actually grouped in one of the blocks, respectively. Thus, in block recursive doubling, 
you need to perform $\mathbf{\bar{y}_{n+N}}+\mathbf{C}\mathbf{\bar{y}_{n}}=\mathbf{\bar{w}_{n+N}}$ by updating the entire output blocks 
$\mathbf{y_{N-2}}$ and $\mathbf{y_{N-1}}$ that include all $\mathbf{\bar{y}}_n$, for example, for the first level,
$$
\begin{aligned}
\mathbf{y}^{(1)}_{\mathbf{N}-2} &= \mathbf{w_{N-2}} - e
\begin{bmatrix}
0\\
y^{(0)}_{N-2} \\
0 \\
w^{(0)}_{3N-2} \\
0 \\
w^{(0)}_{5N-2} \\
0 \\
w^{(0)}_{7N-2}
\end{bmatrix}
- g
\begin{bmatrix}
0 \\
y^{(0)}_{N-1} \\
0 \\ 
w^{(0)}_{3N-1} \\
0 \\
w^{(0)}_{5N-1} \\
0 \\
w^{(0)}_{7N-1}
\end{bmatrix}
\\
\mathbf{y}^{(1)}_{\mathbf{N}-1} &= \mathbf{w_{N-1}} - f
\begin{bmatrix}
0 \\
y^{(0)}_{N-2} \\
0 \\
w^{(0)}_{3N-2} \\
0 \\
w^{(0)}_{5N-2} \\
0 \\
w^{(0)}_{7N-2}
\end{bmatrix}
- d
\begin{bmatrix}
0 \\
y^{(0)}_{N-1} \\
0 \\
w^{(0)}_{3N-1} \\
0 \\
w^{(0)}_{5N-1} \\
0 \\
w^{(0)}_{7N-1}
\end{bmatrix}
\end{aligned}
$$
for the second level,
$$
\mathbf{C}^2 =
\begin{bmatrix}
e^2+gf & eg+gd \\
fe+df & fg+d^2
\end{bmatrix}
$$
$$
\begin{aligned}
\mathbf{y}^{(2)}_{\mathbf{N}-2} &= \mathbf{y}^{(1)}_{\mathbf{N}-2} -
\begin{bmatrix}
0 \\
0 \\
e \\
e^2+gf \\
0 \\
0 \\
e \\
e^2+gf \\
\end{bmatrix}
\begin{bmatrix}
0\\
0 \\
y^{(1)}_{2N-2} \\
y^{(1)}_{2N-2} \\
0 \\
0 \\
y^{(1)}_{6N-2} \\
y^{(1)}_{6N-2}
\end{bmatrix}
-
\begin{bmatrix}
0 \\
0 \\
g \\
eg+gd \\
0 \\
0 \\
g \\
eg+gd \\
\end{bmatrix}
\begin{bmatrix}
0\\
0 \\
y^{(1)}_{2N-1} \\
y^{(1)}_{2N-1} \\
0 \\
0 \\
y^{(1)}_{6N-1} \\
y^{(1)}_{6N-1}
\end{bmatrix}
\\
\mathbf{y}^{(2)}_{\mathbf{N}-1} &= \mathbf{y}^{(1)}_{\mathbf{N}-1} - 
\begin{bmatrix}
0 \\
0 \\
f \\
fe+df \\
0 \\
0 \\
f \\
fe+df \\
\end{bmatrix}
\begin{bmatrix}
0\\
0 \\
y^{(1)}_{2N-2} \\
y^{(1)}_{2N-2} \\
0 \\
0 \\
y^{(1)}_{6N-2} \\
y^{(1)}_{6N-2}
\end{bmatrix}
-
\begin{bmatrix}
0 \\
0 \\
d \\
fg+d^2 \\
0 \\
0 \\
d \\
fg+d^2 \\
\end{bmatrix}
\begin{bmatrix}
0\\
0 \\
y^{(1)}_{2N-1} \\
y^{(1)}_{2N-1} \\
0 \\
0 \\
y^{(1)}_{6N-1} \\
y^{(1)}_{6N-1}
\end{bmatrix}
\end{aligned}
$$
so as the third level and the initial step.

- The computational complexity of recursive doubling is determined by $L$. An important point that needs to be pointed out is that processing $N$ number of blocks only requires one-time recursive doubling. The rest computation is all about non-data-dependent forwarding that is illustrated by $\mathbf{H_{12}}$. Thus, without considering any hardware limitation, increasing $N$ will increase the performance of computing homogeneous solution.

In [94]:
def recursive_doubling_factor(HA):

    # compute C = [e g;f d] and C powers 
    e = HA[0][-2]
    g = HA[1][-2]
    f = HA[0][-1]
    d = HA[1][-1]

    C = np.array([[e,g],[f,d]])

    C_list = [None]*(L//2)

    C_list[0] = C
    for l in range(1,L//2):
        C_list[l] = -C @ C_list[l-1] # this need to be modified in c++, 

    return C_list

In [95]:
def block_recursive_doubling_V8(W,HA,yi2,yi1):

        # C_list: a list include C,C2,C3,C4
        C_list = recursive_doubling_factor(HA)
    
        # a generalized block recursive doubling has been given but super hard to match from code and theory.
        # thus, here we only work on the case when L=8.

        Y = np.zeros_like(W,dtype=float)
        
        # initialization: get \bar{y}_{N-2} ready, y^{(0)}_{N-2} = w_{N-2} + yi2*[C00 0 ...] + yi1*[C01 0 ...]
        Y[-2] = W[-2] - yi2*np.array([C_list[0][0][0],0,0,0,0,0,0,0]) - yi1*np.array([C_list[0][0][1],0,0,0,0,0,0,0])
        Y[-1] = W[-1] - yi2*np.array([C_list[0][1][0],0,0,0,0,0,0,0]) - yi1*np.array([C_list[0][1][1],0,0,0,0,0,0,0])

        # first round: get \bar{y}_{2N-2} ready, y^{(1)}_{N-2} = y^{(0)}_{N-2} + [0 y_{N-2} 0 y_{3N-2}]*[0 C00 ...] + [0 y_{N-1} 0 y_{3N-1}]*[0 C11 ...]
        y_tmp2 = np.array([0, Y[-2][0],0, Y[-2][2],0, Y[-2][4],0, Y[-2][6]])
        y_tmp1 = np.array([0, Y[-1][0],0, Y[-1][2],0, Y[-1][4],0, Y[-1][6]])
        # can be precompute. 22 -> yi2 yi2
        C_tmp22 = np.array([0, C_list[0][0][0],0, C_list[0][0][0],0, C_list[0][0][0],0, C_list[0][0][0]])
        C_tmp21 = np.array([0, C_list[0][0][1],0, C_list[0][0][1],0, C_list[0][0][1],0, C_list[0][0][1]])
        C_tmp12 = np.array([0, C_list[0][1][0],0, C_list[0][1][0],0, C_list[0][1][0],0, C_list[0][1][0]])
        C_tmp11 = np.array([0, C_list[0][1][1],0, C_list[0][1][1],0, C_list[0][1][1],0, C_list[0][1][1]])
        
        Y[-2] = Y[-2] - y_tmp2*C_tmp22 - y_tmp1*C_tmp21
        Y[-1] = Y[-1] - y_tmp2*C_tmp12 - y_tmp1*C_tmp11
        
        # second round: get \bar{y}_{3N-2},\bar{y}_{4N-2} ready, 
        # y^{(2)}_{N-2} = y^{(1)}_{N-2} + [0 0 y_{2N-2} y_{2N-2} 0 0 y_{6N-2} y_{6N-2}]*[0 0 C00 C^200...] + [0 0 y_{2N-1} y_{2N-1} ...
        y_tmp2 = np.array([0, 0, Y[-2][1], Y[-2][1], 0,0, Y[-2][5], Y[-2][5]])
        y_tmp1 = np.array([0, 0, Y[-1][1], Y[-1][1], 0,0, Y[-1][5], Y[-1][5]])
        # can be precompute. 22 -> yi2 yi2
        C_tmp22 = np.array([0, 0, C_list[0][0][0], C_list[1][0][0], 0, 0, C_list[0][0][0], C_list[1][0][0]])
        C_tmp21 = np.array([0, 0, C_list[0][0][1], C_list[1][0][1], 0, 0, C_list[0][0][1], C_list[1][0][1]])
        C_tmp12 = np.array([0, 0, C_list[0][1][0], C_list[1][1][0], 0, 0, C_list[0][1][0], C_list[1][1][0]])
        C_tmp11 = np.array([0, 0, C_list[0][1][1], C_list[1][1][1], 0, 0, C_list[0][1][1], C_list[1][1][1]])
        
        Y[-2] = Y[-2] - y_tmp2*C_tmp22 - y_tmp1*C_tmp21
        Y[-1] = Y[-1] - y_tmp2*C_tmp12 - y_tmp1*C_tmp11

        # last round: get \bar{y}_{5N-2},\bar{y}_{6N-2},\bar{y}_{7N-2},\bar{y}_{8N-2} ready,
        # y^{(3)}_{N-2} = y^{(2)}_{N-2} + [0 0 0 0 y_{4N-2} y_{4N-2} y_{4N-2} y_{4N-2}]*[0 0 0 0 C00 C^200 C^300 C^400] + [0 0 0 0 y_{4N-1} y_{4N-1} ...
        y_tmp2 = np.array([0, 0, 0, 0, Y[-2][3], Y[-2][3], Y[-2][3], Y[-2][3]])
        y_tmp1 = np.array([0, 0, 0, 0, Y[-1][3], Y[-1][3], Y[-1][3], Y[-1][3]])
        # can be precompute. 22 -> yi2 yi2
        C_tmp22 = np.array([0, 0, 0, 0, C_list[0][0][0], C_list[1][0][0], C_list[2][0][0], C_list[3][0][0]])
        C_tmp21 = np.array([0, 0, 0, 0, C_list[0][0][1], C_list[1][0][1], C_list[2][0][1], C_list[3][0][1]])
        C_tmp12 = np.array([0, 0, 0, 0, C_list[0][1][0], C_list[1][1][0], C_list[2][1][0], C_list[3][1][0]])
        C_tmp11 = np.array([0, 0, 0, 0, C_list[0][1][1], C_list[1][1][1], C_list[2][1][1], C_list[3][1][1]])
        
        Y[-2] = Y[-2] - y_tmp2*C_tmp22 - y_tmp1*C_tmp21
        Y[-1] = Y[-1] - y_tmp2*C_tmp12 - y_tmp1*C_tmp11
    
        return Y

##### Forwarding the first $\mathbf{N}{-}2$ blocks
After computing the last 2 blocks by recursive doubling, the first $\mathbf{N}{-}2$ blocks, which is relavant to $\mathbf{H_{12}}$ can be computed by
$$
\mathbf{y_{n}} +  \mathbf{u_{2,n}}\mathbf{y_{N-2}} + \mathbf{u_{1,n}}\mathbf{y_{N-1}} + \mathbf{q_{2,n}}y_{-2} + \mathbf{q_{1,n}}y_{-1} = \mathbf{w_{n}}
$$
and finally it can be easily reduced to
$$
\mathbf{y_{n}} +  h_{2,n}\mathbf{y_{-2}} + h_{1,n}\mathbf{y_{-1}}  = \mathbf{w_{n}}
$$
Note, the forwarding computation is all non-data-dependent.

In [97]:
def ph_factors(a1,a2):

    # section of block filtering factor
    # impulse response of recursive part, h0=[s_0,s_1...]
    h0 = np.zeros(N,dtype=float)
    h0[0] = 1
    h0[1] = -a1*h0[0]
    for n in range(2,N):
        h0[n] = -a1*h0[n-1] - a2*h0[n-2]

    # filtering matrix HA for x_{n-1}
    HA = np.zeros((2,N),dtype=float)
    HA[0] = a2*h0
    HA[1] = a1*h0 + a2*np.concatenate(([0],h0[:-1]))
    
    return HA

In [98]:
class homogeneous_solution:

    def __init__(self,a1,a2,yi1,yi2):

        # scalar coefficient of u and q
        self.HA = ph_factors(a1,a2)
        _,self.buffer_y=start_register(yi1=yi1,yi2=yi2)
        self.a1=a1
        self.a2=a2

    def forward(self,W,Y):

        yvi2 = np.concatenate(([self.buffer_y[-2]],Y[-2][:-1]),dtype=float)
        yvi1 = np.concatenate(([self.buffer_y[-1]],Y[-1][:-1]),dtype=float)

        for n in range(len(Y)-2):
            Y[n] = W[n] - self.HA[0][n]*yvi2 - self.HA[1][n]*yvi1

        self.buffer_y[-2] = Y[-2][-1]
        self.buffer_y[-1] = Y[-1][-1]
            
        return Y

    def __call__(self,W):

        Y = block_recursive_doubling_V8(W,self.HA,self.buffer_y[-2],self.buffer_y[-1])
        Y = self.forward(W,Y)

        return Y

In [99]:
BF = block_filtering(b1,b2,a1,a2,xi1,xi2,yi1,yi2)

In [100]:
FIR = non_recursive_solution(b1,b2,xi1,xi2)
HS = homogeneous_solution(a1,a2,yi1,yi2)

Xp = permute_V8(X)
V = FIR(Xp)
W = particular_solution(V,a1,a2)
Yp = HS(W)
y_ph = depermute_V8(Yp)

assert np.isclose(np.ravel(y_ph),y_bench).all()
print('ph decomposition ok')

ph decomposition ok


### Solution II for Recursive Part: Cyclic Reduction
The core idea is to recursively eliminate half of the systems at each round.
the first round involves a permutation of the unknowns into even- and odd-indexed blocks,
$$
\left[
\begin{array}{cccc:cccc}
 \mathbf{I} &   &   & a_{2}\mathbf{R}  &   &   &   &  a_{1}\mathbf{R} \\
 a_{2}\mathbf{I} &  \mathbf{I} &   &   & a_{1}\mathbf{I}  &   &   &   \\
  & \ddots  & \ddots  &   &   & \ddots  &   &   \\
  &   & a_{2}\mathbf{I}  & \mathbf{I}  &   &   & a_{1}\mathbf{I}  &   \\
\hdashline
 a_{1}\mathbf{I} &   &   &   & \mathbf{I}  &   &   & a_{2}\mathbf{R}  \\
  & a_{1}\mathbf{I}  &   &   & a_{2}\mathbf{I}  & \mathbf{I}  &   &   \\
  &   &  \ddots &   &   & \ddots  & \ddots  &   \\
  &   &   &  a_{1}\mathbf{I} &   &   & a_{2}\mathbf{I}  & \mathbf{I} 
\end{array}
\right]
\begin{bmatrix}
 \mathbf{y_{0}} \\ \mathbf{y_{2}} \\ \vdots \\ \mathbf{y_{N-2}} \\
\hdashline
 \mathbf{y_{1}} \\ \mathbf{y_{3}} \\ \vdots \\ \mathbf{y_{N-1}} \\ 
\end{bmatrix}
=
\begin{bmatrix}
 \mathbf{x_{0}} \\ \mathbf{x_{2}} \\ \vdots \\ \mathbf{x_{N-2}} \\
\hdashline
 \mathbf{x_{1}} \\ \mathbf{x_{3}} \\ \vdots \\ \mathbf{x_{N-1}} \\ 
\end{bmatrix}
$$
Apply Gaussian elimination to transform the upper-left part to an identity matrix, and then compute its Schur complement, 
$$
\left[
\begin{array}{cccc:cccc}
 \mathbf{I} &   &   &   &   &   & d^{(i)}\mathbf{R}  & c^{(i)}\mathbf{R}  \\
  &  \mathbf{I} &   &   & c^{(i)}\mathbf{I}  &   &   & d^{(i)}\mathbf{R}  \\
 &   & \ddots  &   &   d^{(i)}\mathbf{I} & \ddots  &   &   \\
  &   &   & \mathbf{I}  &   &  \ddots & c^{(i)}\mathbf{I}  &   \\
\hdashline
  &   &   &   & \mathbf{I}  &   & f^{(i)}\mathbf{R}  & e^{(i)}\mathbf{R}  \\
  &   &   &   & e^{(i)}\mathbf{I}  & \mathbf{I}  &   & f^{(i)}\mathbf{R}  \\
  &   &   &   & f^{(i)}\mathbf{I}  & \ddots  & \ddots  &   \\
  &   &   &   &   & \ddots  & e^{(i)}\mathbf{I}  & \mathbf{I} 
\end{array}
\right]
\begin{bmatrix}
 \mathbf{y}_{2^{i}-1} \\[2pt]
 \mathbf{y}_{3\cdot2^{i}-1} \\[4pt]
 \vdots \\[2pt]
 \mathbf{y}_{\mathbf{N}-2^{i}-1} \\
\hdashline
 \mathbf{y}_{2\cdot2^{i}-1} \\[2pt]
 \mathbf{y}_{4\cdot2^{i}-1} \\[4pt]
 \vdots \\[4pt]
 \mathbf{y}_{\mathbf{N}-1} \\ 
\end{bmatrix}
{=}
\begin{bmatrix}
\mathbf{x}^{(i)}_{2^{i}{-}1}  \\[2pt]
 \mathbf{x}^{(i)}_{3\cdot2^{i}{-}1} \\[1pt]
 \vdots \\
 \mathbf{x}^{(i)}_{\mathbf{N}-2^{i}{-}1} \\
\hdashline
 \mathbf{x}^{(i)}_{2\cdot2^{i}-1} \\[2pt]
 \mathbf{x}^{(i)}_{4\cdot2^{i}-1} \\[1pt]
 \vdots \\[2pt]
 \mathbf{x}^{(i)}_{\mathbf{N}-1} \\ 
\end{bmatrix}
$$
where $i{=}1,2,\cdots,\log_{2}\text{N}$ denotes the number of reduced steps, and
$$
\begin{aligned}
f^{(0)} &= a_{2} \\
e^{(0)} &= a_{1} \\
d^{(i)} &= -\frac{f^{(i-1)}f^{(i-1)}}{e^{(i-1)}} \\
c^{(i)} &= e^{(i-1)}-\frac{f^{(i-1)}}{e^{(i-1)}} \\
f^{(i)} &= f^{(i-1)}f^{(i-1)} \\
e^{(i)} &= 2f^{(i-1)}-e^{(i-1)}e^{(i-1)}. \\
\end{aligned}
$$
The upper-half blocks on the right-hand side are computed by
$$
\begin{aligned}
    \mathbf{x}^{(i)}_{2^{i}{-}1} &= \mathbf{x}^{(i-1)}_{2^{i{-}1}{-}1} - \frac{f^{(i-1)}}{e^{(i-1)}}\mathbf{R}\mathbf{x}^{(i-1)}_{\mathbf{N}{-}1} \\
    \mathbf{x}^{(i)}_{(2n+1)\cdot2^{i}{-}1} &= \mathbf{x}^{(i-1)}_{(2n+1)\cdot2^{i{-}1}{-}1} - \frac{f^{(i-1)}}{e^{(i-1)}}\mathbf{x}^{(i-1)}_{2n\cdot2^{i{-}1}{-}1}
\end{aligned}
$$
and the lower half blocks,
$$
\mathbf{x}^{(i)}_{(2n)\cdot2^{i}{-}1} = \mathbf{x}^{(i-1)}_{2n\cdot2^{i{-}1}{-}1} - e^{(i-1)}\mathbf{x}^{(i)}_{(2n-1)\cdot2^{i}{-}1}
$$

The reduction of system follows the recursive structure above until the last two rounds. For the second last round, the reduced system becomes
$$
\mathbf{A}^{(j)}_{\mathbf{CR}} = 
\left[
\begin{array}{c:c}
\begin{array}{cc}
\mathbf{I} &  \\
 & \mathbf{I}
\end{array}
&
\begin{array}{cc}
d^{(j)}\mathbf{R} & c^{(j)}\mathbf{R} \\
c^{(j)}\mathbf{I} & d^{(j)}\mathbf{R}
\end{array}
\\
\hdashline
\begin{array}{cc}
 &  \\
 & 
\end{array}
&
\begin{array}{cc}
\mathbf{I}+f^{(j)}\mathbf{R} & e^{(j)}\mathbf{R} \\
e^{(j)}\mathbf{I} & \mathbf{I}{+}f^{(j)}\mathbf{R}
\end{array}
\end{array}
\right]
$$
where $j=\log_{2}\text{N}{-}1$. For the last round,
$$
\left[
\begin{array}{cc}
\mathbf{I}  & c^{(l)}\mathbf{R} + d^{(l)}\mathbf{R}^2 \\
\mathbf{0} & \mathbf{I} + e^{(l)}\mathbf{R} + f^{(l)}\mathbf{R}^2
\end{array}
\right]
$$
To compute the output blocks in backward, the first step is one-time block filtering.

#### Adding Initial States in Cyclic Reduction
The above discussion focus more on the recursive pattern of reducing system in cyclic reduction. Next, we discuss about adding the initial states.
After permutation and Gaussian elimination, the recursive pattern adding the initial states is given by
$$
\mathbf{A}^{(i)}_{\mathbf{CR}}\mathbf{Y}^{(i)}_{\mathbf{CR}} = \mathbf{X}^{(i)}_{\mathbf{CR}} - 
\begin{bmatrix}
 h^{(i-1)}\mathbf{s}  & g^{(i-1)}\mathbf{s} \\
   & d^{(i)}\mathbf{s} \\
   & \\
   & \\
\hdashline
 h^{(i)}\mathbf{s}  & g^{(i)}\mathbf{s} \\
   & f^{(i)}\mathbf{s} \\
   & \\
   & \\
\end{bmatrix}
\begin{bmatrix}
y_{-2} \\
y_{-1}
\end{bmatrix}
$$
where
$$
\begin{aligned}
h^{(0)} &= a_{2} \\
g^{(0)} &= a_{1} \\
h^{(i)} &= -e^{(i-1)}h^{(i-1)} \\
g^{(i)} &= f^{(i-1)}-e^{(i-1)}g^{(i-1)} \\
\end{aligned}
$$
The recursive pattern is maintained until the last round, and in the last round,
$$
\mathbf{A}^{(j)}_{\mathbf{CR}}\mathbf{Y}^{(j)}_{\mathbf{CR}} = \mathbf{X}^{(j)}_{\mathbf{CR}} - 
\begin{bmatrix}
 h^{(j-1)}\mathbf{s}  & (g^{(j-1)}{+}d^{(j)}\mathbf{R})\mathbf{s} \\
 h^{(j)}\mathbf{s}  & (g^{(j)}{+}f^{(j)}\mathbf{R})\mathbf{s} \\
\end{bmatrix}
\begin{bmatrix}
y_{-2} \\
y_{-1}
\end{bmatrix}
$$

In [103]:
def cyclic_reduction_factor(a1,a2):

    rounds = np.log2(N).astype(int) 
    
    f = np.zeros(rounds+1,dtype=float)
    e = np.zeros(rounds+1,dtype=float)
    d = np.zeros(rounds+1,dtype=float)
    c = np.zeros(rounds+1,dtype=float)
    h = np.zeros(rounds+1,dtype=float) 
    g = np.zeros(rounds+1,dtype=float)
    
    f[0] = a2
    e[0] = a1
    h[0] = a2
    g[0] = a1
    for n in range(1,rounds+1):
        
        d[n] = -f[n-1]**2/e[n-1]
        c[n] = e[n-1] - f[n-1]/e[n-1]
        f[n] = -e[n-1]*d[n]
        e[n] = f[n-1] - e[n-1]*c[n]
        h[n] = -e[n-1]*h[n-1]
        g[n] = f[n-1] - e[n-1]*g[n-1]

    return f,e,d,c,h,g

In [104]:
def cr_block_filtering_factors(e,f,h,g):

    # the first column of inverse matrix (I+eR+fR^2)^-1
    h0 = np.zeros(L,dtype=float)
    h0[0] = 1
    h0[1] = -e*h0[0]
    for n in range(2,L):
        h0[n] = -e*h0[n-1] - f*h0[n-2]

    # filtering matrix H for x_n
    H = np.zeros((L,L),dtype=float)
    H[0] = h0
    for n in range(1,L):
        H[n] = np.concatenate(([0],H[n-1][:-1])) # shuffle

    # filtering matrix HA for y_{n-1}
    HA = np.zeros((2,L),dtype=float)
    HA[0] = h*h0
    HA[1] = g*h0 + f*np.concatenate(([0],h0[:-1]))

    return H,HA

In [105]:
class cyclic_reduction:
    
     def __init__(self,a1,a2,yi1,yi2):

        self.f,self.e,self.d,self.c,self.h,self.g = cyclic_reduction_factor(a1,a2)  
        self.H,self.HA = cr_block_filtering_factors(self.e[-1],self.f[-1],self.h[-1],self.g[-1])
        _,self.buffer_y=start_register(yi1=yi1,yi2=yi2)
        
     def forward(self,X,ro=0,ind=1):
    
        # gaussian elimination for X
        k = 2**(ro+1)
        
        if ro <= np.log2(N):
            
            # top side
            for n in range(N//k):            
                if n == 0:
                    X[k*n+k//2-ind] -= self.f[ro]/self.e[ro]*np.concatenate(([0],X[-ind][:-1]),dtype=float) 
                else:
                    X[k*n+k//2-ind] -= self.f[ro]/self.e[ro]*X[k*n-ind]
                    
            # bottom side
            for n in range(N//k):
                X[k*n+k-ind] -= self.e[ro]*X[k*n+k//2-ind]
            
            ro += 1
            return self.forward(X,ro,ind)
            
        else:
            return X

     def backward(self,X):

        # compute output blocks
        Y = np.zeros_like(X,dtype=float)
        rounds = np.log2(N).astype(int) 

        # block filtering to get the last block
        for l in range(L):
            Y[-1] += self.H[l]*X[-1][l]

        Y[-1] = Y[-1] - self.HA[0]*self.buffer_y[-2] - self.HA[1]*self.buffer_y[-1]

        # compute the rest blocks        
        for ro in range(rounds-1,-1,-1):
            
            k = 2**(ro+1)
            for n in range(2**(rounds-1-ro)):
                
                # the block before last, y[N/2-1] + (cR+dR^2)y[N-1] = x[N/2-1] - hy_{-2} - (g+dR)y_{-1} 
                if ro == rounds-1:
                    # hy_{-2} and cRy[N-1] in same block
                    tmp_i2 = np.concatenate(([self.h[ro]],np.ones(L-1,dtype=float)*self.c[ro+1]))*np.concatenate(([self.buffer_y[-2]],Y[-1][:-1]))
                    # (g+dR)y_{-1} and dR^2y[N-1] in same block
                    tmp_i1 = np.concatenate(([self.g[ro]],np.ones(L-1,dtype=float)*self.d[ro+1]))*np.concatenate(([self.buffer_y[-1],self.buffer_y[-1]],Y[-1][:-2])) 
                    Y[k//2-1] = X[k//2-1] - tmp_i2 - tmp_i1
                    
                # forwarding the rest in pattern
                else:
                    if n == 0:
                        Y[k//2-1] = X[k//2-1] - np.concatenate(([self.h[ro]],np.ones(L-1,dtype=float)*self.d[ro+1]))*np.concatenate(([self.buffer_y[-2]],Y[2**rounds-k-1][:-1])) - np.concatenate(([self.g[ro]],np.ones(L-1,dtype=float)*self.c[ro+1]))*np.concatenate(([self.buffer_y[-1]],Y[-1][:-1])) 
                    elif n == 1:
                        Y[k+k//2-1] = X[k+k//2-1] - self.c[ro+1]*Y[k-1] - self.d[ro+1]*np.concatenate(([self.buffer_y[-1]],Y[-1][:-1]))
                    else:
                        Y[k*n+k//2-1] = X[k*n+k//2-1] - self.c[ro+1]*Y[k*n-1] - self.d[ro+1]*Y[k*(n-1)-1]

        self.buffer_y[-2] = Y[-2][-1]
        self.buffer_y[-1] = Y[-1][-1]

        return Y

     def __call__(self,X):

        W = self.forward(X)
        Y = self.backward(W)

        return Y

In [106]:
FIR = non_recursive_solution(b1,b2,xi1,xi2)
CR = cyclic_reduction(a1,a2,yi1,yi2)

Xp = permute_V8(X)
V = FIR(Xp)
Yp = CR(V)
y_cr = depermute_V8(Yp)

assert np.isclose(np.ravel(y_cr),y_bench).all()
print('cyclic reduction ok')

cyclic reduction ok
